In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import pickle
import numpy as np
import os

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
import cv2
import numpy as np
import os
import pickle
from sklearn.neighbors import KNeighborsClassifier

# Function to collect training data for a specific person
def collect_training_data(video_capture, faces_detector, name, max_samples=100):
    """
    Collects training data for a specific person.
    
    Parameters:
        video_capture (cv2.VideoCapture): Video capture object.
        faces_detector (cv2.CascadeClassifier): Cascade classifier for face detection.
        name (str): Name of the person.
        max_samples (int): Maximum number of training samples to collect (default is 100).
    """
    faces_data = []
    i = 0
    
    while True:
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faces_detector.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            crop_img = frame[y:y+h, x:x+w, :]
            if not crop_img.size == 0:
                resized_img = cv2.resize(crop_img, (50, 50))
                if len(faces_data) < max_samples:
                    faces_data.append(resized_img)
        
        i += 1
        cv2.putText(frame, str(len(faces_data)), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Collecting Training Data", frame)
        k = cv2.waitKey(1)
        
        if k == ord('q') or len(faces_data) == max_samples:
            break
    
    faces_data = np.array(faces_data)
    faces_data = faces_data.reshape(len(faces_data), -1)
    
    # Save or update training data for the specific person
    names_file_path = 'datas/names.pkl'
    faces_data_file_path = 'datas/faces_data.pkl'
    
    if 'names.pkl' not in os.listdir('datas/'):
        labels = [name] * len(faces_data)
        with open(names_file_path, 'wb') as f:
            pickle.dump(labels, f)
    else:
        with open(names_file_path, 'rb') as f:
            labels = pickle.load(f)
        labels += [name] * len(faces_data)
        with open(names_file_path, 'wb') as f:
            pickle.dump(labels, f)
    
    if 'faces_data.pkl' not in os.listdir('datas/'):
        with open(faces_data_file_path, 'wb') as f:
            pickle.dump(faces_data, f)
    else:
        with open(faces_data_file_path, 'rb') as f:
            existing_faces_data = pickle.load(f)
        faces_data = np.concatenate((existing_faces_data, faces_data), axis=0)
        with open(faces_data_file_path, 'wb') as f:
            pickle.dump(faces_data, f)


In [4]:

# Function to recognize faces
def recognize_faces(video_capture, faces_detector, knn_classifier):
    """
    Recognizes faces in real-time using a trained classifier.
    
    Parameters:
        video_capture (cv2.VideoCapture): Video capture object.
        faces_detector (cv2.CascadeClassifier): Cascade classifier for face detection.
        knn_classifier (sklearn.neighbors.KNeighborsClassifier): Trained KNN classifier.
    """
    while True:
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faces_detector.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            crop_img = frame[y:y+h, x:x+w, :]
            if not crop_img.size == 0:
                resized_img = cv2.resize(crop_img, (50, 50)).flatten().reshape(1, -1)
                output = knn_classifier.predict(resized_img)
                cv2.putText(frame, str(output[0]), (x, y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255,0), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        cv2.imshow("Recognizing Faces", frame)
        k = cv2.waitKey(1)
        
        if k == ord('q'):
            break

# Main function



In [5]:
def main():
    video_capture = cv2.VideoCapture(0)
    faces_detector = cv2.CascadeClassifier('datas/haarcascade_frontalface_default.xml')
    
    # Train or update the model for each person
    num_persons = int(input("Enter the number of persons: "))
    for _ in range(num_persons):
        name = input("Enter the person's name: ")
        collect_training_data(video_capture, faces_detector, name)
    
    # Load training data
    with open('datas/names.pkl', 'rb') as f:
        labels = pickle.load(f)
    with open('datas/faces_data.pkl', 'rb') as f:
        faces_data = pickle.load(f)
    
    # Train KNN classifier
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    knn_classifier.fit(faces_data, labels)
    
    # Recognize faces
    recognize_faces(video_capture, faces_detector, knn_classifier)
    
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Enter the number of persons:  2
Enter the person's name:  Madhavan
Enter the person's name:  Virat
